In [1]:
import networkx as nx
import numpy as np
from matplotlib import pyplot as plt
from functools import partial as part
import random

er_fast = nx.fast_gnp_random_graph


In [2]:
def er_fast_Nk(N, kavg, seed=123456):
    p = kavg/(N-1)
    return er_fast(N, p, seed=seed)


In [9]:
def degplot(degs, *args, plotter=plt, **kwargs):
    km, kM = min(degs), max(degs)
    plotter.plot(range(km, kM+1), (np.bincount(degs)/len(degs))
                 [km:kM+1], *args, **kwargs)


def degplot4rng(rng_gnr, *args, trials=10, seed=123456, plotter=plt, **kwargs):
    rngs = map(rng_gnr, range(seed, seed+trials))
    degs = [k for g in rngs for n, k in g.degree]
    degplot(degs, *args, plotter=plotter, **kwargs)


def degplot4er(N, kavg, *args, trials=10, seed=123456, plotter=plt, **kwargs):
    degplot4rng(part(er_fast_Nk, N, kavg), *args, trials=trials,
                seed=seed, plotter=plotter, **kwargs)


In [6]:
def aspl_estim(g: nx.Graph, nsample=10):
    N = g.number_of_nodes()
    spls = []
    for _ in range(nsample):
        ps = nx.shortest_path_length(g, source=random.randint(0, N-1))
        spls.append(np.mean(tuple(ps.values())))
    return np.mean(spls)


def aspl4er(N, kavg, trials=10, seed=123456, nsample=False):
    """Average shortest path length for ER networks"""
    graphs = [er_fast_Nk(N, kavg, seed=seed+t) for t in range(trials)]
    ls = [aspl_estim(g, nsample=nsample)
          if nsample else nx.average_shortest_path_length(g) for g in graphs]
    lBar = np.mean(ls)
    lstd = np.std(ls)
    l = np.log(N)/np.log(kavg)
    l1 = (np.log(N)-0.57722)/np.log(kavg)+1/2
    return (lBar, lstd), (l, l1)


In [ ]:
degplot4er(100, 20, 'o', seed=123456)
plt.ylabel("$P_k$")
plt.xlabel("$k$")
plt.show()
